<a href="https://colab.research.google.com/github/Jonathan9219/ANGULAR/blob/master/Accidentalidad_Vial_Antioquia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [58]:
# CELDA 1: instalar dependencias y limitar hilos
!pip install -q sqlalchemy matplotlib seaborn scikit-learn joblib
# CELDA A: instalar dependencias para Streamlit + ngrok (Colab)
!pip install -q streamlit plotly sqlalchemy streamlit-aggrid pyngrok

# CELDA C: exponer la app con pyngrok y ejecutar streamlit
import subprocess, time, os, signal
import os, warnings, matplotlib
from pyngrok import ngrok
warnings.filterwarnings("ignore")

# Control de hilos (para evitar sobrecarga en Colab)
os.environ["OMP_NUM_THREADS"] = "2"
os.environ["OPENBLAS_NUM_THREADS"] = "2"
os.environ["MKL_NUM_THREADS"] = "2"
os.environ["NUMEXPR_NUM_THREADS"] = "2"

# Forzar backend no interactivo
matplotlib.use("Agg")

print("Entorno configurado. Dependencias instaladas y threads limitados.")

Entorno configurado. Dependencias instaladas y threads limitados.


In [59]:
# CELDA 2: importaciones y rutas
import re, warnings, os
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
import seaborn as sns
import joblib

from sqlalchemy import create_engine
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, confusion_matrix, roc_curve, classification_report
)

warnings.filterwarnings("ignore")
sns.set_theme(style="whitegrid", context="notebook")

# Rutas y constantes (ajustadas a Colab)
CSV_PATH = "https://raw.githubusercontent.com/ShinnetValllejo/Accidentalidad_Vial_Antioquia/main/AMVA_Accidentalidad_20191022_2.csv"
SEPARATOR = ";"
ENCODING = "latin-1"

DB_PATH = Path("/content/Proyecto_Accidentalidad_Vial_Antioquia.db")
TABLE_NAME = "Accidentalidad_Vial_Antioquia"
OUT_DIR = Path("/content/Graficas_Salida")
MODEL_DIR = Path("/content/Modelo_Predict")

OUT_DIR.mkdir(parents=True, exist_ok=True)
MODEL_DIR.mkdir(parents=True, exist_ok=True)

PALETTE_GREEN = ["#137598","#026937", "#f9a12c", "#8dc63f", "#3ebdac","#70205b","#ef434d"]
print("Rutas y entorno listos.")

Rutas y entorno listos.


In [60]:
# CELDA 3: funciones de limpieza (idénticas a tu script)
def clean_fecha(fecha):
    if pd.isna(fecha):
        return None
    s = str(fecha).strip()
    match = re.search(r"\d{1,2}/\d{1,2}/\d{2,4}", s)
    if not match:
        return None
    for fmt in ["%d/%m/%Y", "%m/%d/%Y"]:
        try:
            return pd.to_datetime(match.group(0), format=fmt).strftime("%d/%m/%Y")
        except:
            continue
    return None

def clean_hora(hora):
    if pd.isna(hora):
        return None
    s = re.sub(r"\s+", " ", str(hora).strip().replace("\u00A0", " "))
    m = re.search(r"(\d{1,2}:\d{2}(:\d{2})?)", s)
    s = m.group(1) if m else s
    s = s.replace("p m", "PM").replace("pm", "PM").replace("a m", "AM").replace("am", "AM")
    return s.strip()

def try_parse_time(val):
    if pd.isna(val):
        return None
    for fmt in ["%I:%M:%S %p", "%I:%M %p", "%H:%M:%S", "%H:%M"]:
        t = pd.to_datetime(val, format=fmt, errors="coerce")
        if pd.notna(t):
            return t
    # fallback: try pandas generic parse
    t = pd.to_datetime(val, errors="coerce", dayfirst=True)
    return t if pd.notna(t) else None

def clasificar_jornada(hora_str):
    if pd.isna(hora_str):
        return None
    try:
        h = int(hora_str.split(":")[0])
        if 0 <= h < 6: return "MADRUGADA"
        if 6 <= h < 12: return "MAÑANA"
        if 12 <= h < 18: return "TARDE"
        if 18 <= h < 24: return "NOCHE"
    except:
        return None

In [61]:
# CELDA 4: carga y limpieza completa (reproduce tu CargaLimpiezaBD.py)
print("Cargando CSV desde GitHub...")
df = pd.read_csv(CSV_PATH, sep=SEPARATOR, encoding=ENCODING, low_memory=False)
print(f"Datos cargados: {df.shape[0]} filas x {df.shape[1]} columnas")

# Normalizar columnas
df.columns = df.columns.str.strip()
rename_map = {
    "GRAVEDAÑOSSADAÑOSS": "GRAVEDAD_ACCIDENTE",
    "DÍA DE LA SEMANA": "NOM_DIA_SEMANA",
    "DIA DE LA SEMANA": "NOM_DIA_SEMANA"
}
df.rename(columns=rename_map, inplace=True)

# Aplicar limpieza a FECHA y HORA
df["FECHA"] = df["FECHA"].astype(str).map(clean_fecha)
df["HORA"] = df["HORA"].astype(str).map(clean_hora)

# Crear HORA_dt y NUM_HORA
df["HORA_dt"] = df["HORA"].apply(try_parse_time)
df["NUM_HORA"] = df["HORA_dt"].apply(
    lambda t: (t.hour + t.minute / 60.0 + t.second / 3600.0) if pd.notna(t) else None
)
df['hora_redondeada'] = df['HORA_dt'].dt.floor('h')
df["HORA"] = df["HORA_dt"].dt.strftime("%H:%M:%S")
df.drop(columns=["HORA_dt"], inplace=True)

# Clasificar jornada
df["JORNADA"] = df["HORA"].map(clasificar_jornada)

# FECHA_dt -> NUM_DIA_SEMANA y NUM_MES
df["FECHA_dt"] = pd.to_datetime(df["FECHA"], format="%d/%m/%Y", errors="coerce")
df["NUM_DIA_SEMANA"] = df["FECHA_dt"].dt.weekday + 1
df["NUM_MES"] = df["FECHA_dt"].dt.month
df["AÑO"] = df["FECHA_dt"].dt.year
df.drop(columns=["FECHA_dt"], inplace=True)

# Normalización de textos
for col in df.columns:
    if col in ("NUM_HORA", "NUM_DIA_SEMANA", "NUM_MES"):
        continue
    if df[col].dtype == "object":
        df[col] = (
            df[col].astype(str)
                   .str.strip()
                   .str.upper()
                   .str.replace(r"\s+", " ", regex=True)
        )

# Validaciones finales
df["FECHA"] = df["FECHA"].replace("NAN", None)
df["HORA"] = df["HORA"].replace("NAN", None)
df["NUM_HORA"] = pd.to_numeric(df["NUM_HORA"], errors="coerce")

# Guardar en SQLite
engine = create_engine(f"sqlite:///{DB_PATH}")
df.to_sql(TABLE_NAME, con=engine, if_exists="replace", index=False)
print("Guardado en SQLite:", DB_PATH)

# Reporte de nulos por columna (imprime top)
print("\n=== VALIDACIÓN DE NULOS POR CAMPO (muestra top 20) ===")
nulos_por_columna = df.isna().sum().sort_values(ascending=False)
total = len(df)
for col, nulos in nulos_por_columna.head(20).items():
    pct = (nulos / total) * 100
    print(f"{col:<30} -> {nulos:>6} nulos ({pct:5.2f}%)")
print("======================================\n")

Cargando CSV desde GitHub...
Datos cargados: 203435 filas x 11 columnas
Guardado en SQLite: /content/Proyecto_Accidentalidad_Vial_Antioquia.db

=== VALIDACIÓN DE NULOS POR CAMPO (muestra top 20) ===
HORA                           ->      1 nulos ( 0.00%)
NUM_HORA                       ->      1 nulos ( 0.00%)
hora_redondeada                ->      1 nulos ( 0.00%)
COD_MUNICIPIO                  ->      0 nulos ( 0.00%)
MUNICIPIO                      ->      0 nulos ( 0.00%)
CLASE                          ->      0 nulos ( 0.00%)
DIRECCIÓN                      ->      0 nulos ( 0.00%)
FECHA                          ->      0 nulos ( 0.00%)
NOM_DIA_SEMANA                 ->      0 nulos ( 0.00%)
BARRIO                         ->      0 nulos ( 0.00%)
GRAVEDAD_ACCIDENTE             ->      0 nulos ( 0.00%)
DISEÑO                         ->      0 nulos ( 0.00%)
COMUNA                         ->      0 nulos ( 0.00%)
JORNADA                        ->      0 nulos ( 0.00%)
NUM_DIA_SEMANA   

In [62]:
# CELDA 5: utilidades para gráficas y carga desde DB
def paleta_antioquia(n: int):
    if n <= 0: return []
    return PALETTE_GREEN[:n] if n <= len(PALETTE_GREEN) else sns.blend_palette(PALETTE_GREEN, n_colors=n)

def save_fig(fig, path: Path):
    fig.savefig(path, format="jpg", bbox_inches="tight", dpi=300)
    plt.close(fig)

def num_fmt(v): return f"{int(v):,}"

def txt_color(rgb):
    return "black" if (0.299*rgb[0] + 0.587*rgb[1] + 0.114*rgb[2]) > 0.65 else "white"

def load_table(db_path: Path, table: str) -> pd.DataFrame:
    if not db_path.exists():
        raise FileNotFoundError(f"No existe la base de datos: {db_path}")
    with create_engine(f"sqlite:///{db_path}").connect() as conn:
        return pd.read_sql(f"SELECT * FROM {table}", conn)

def format_torta(series: pd.Series, title: str, path: Path):
    colors = paleta_antioquia(len(series)) or paleta_antioquia(1)
    fig, ax = plt.subplots(figsize=(8, 6))
    ax.pie(series.values, labels=series.index, autopct="%1.1f%%", startangle=60,
           colors=colors[::-1], textprops={"fontsize": 12})
    ax.set_title(title, fontsize=18, fontweight="bold")
    save_fig(fig, path)

def format_barra(series: pd.Series, title: str, xlabel: str, ylabel: str, path: Path):
    fig, ax = plt.subplots(figsize=(12, 6))
    series = series.dropna()
    if series.empty:
        ax.set(title=title, xlabel=xlabel, ylabel=ylabel)
        save_fig(fig, path)
        return
    values = series.values.astype(float)
    palette = paleta_antioquia(len(series))
    total, max_w = values.sum() or 1, values.max() or 1
    sns.barplot(x=values, y=series.index, palette=palette, ax=ax)
    for p, val in zip(ax.patches, values):
        w, y = p.get_width(), p.get_y() + p.get_height()/2
        pct, rel = (val/total)*100, w/max_w
        color = txt_color(p.get_facecolor()[:3]) if rel >= 0.12 else "black"
        ax.text(w*0.5 if rel >= 0.12 else w+(max_w*0.01), y,
                f"{num_fmt(val)} ({pct:.1f}%)",
                ha="center" if rel >= 0.12 else "left", va="center",
                fontweight="bold", fontsize=11, color=color)
    ax.set(title=title, xlabel=xlabel, ylabel=ylabel)
    ax.grid(True, linestyle="--", linewidth=0.7, alpha=0.6)
    fig.tight_layout()
    save_fig(fig, path)

In [63]:
df

,COD_MUNICIPIO,MUNICIPIO,FECHA,HORA,NOM_DIA_SEMANA,CLASE,DIRECCIÓN,GRAVEDAD_ACCIDENTE,BARRIO,COMUNA,DISEÑO,NUM_HORA,hora_redondeada,JORNADA,NUM_DIA_SEMANA,NUM_MES,AÑO
0,79,BARBOSA,03/01/2015,01:00:00,SABADO,CHOQUE,VIA BELLO HATILLO KM16+200,HERIDOS,EL HATILLO,SIN INFORMACIÓN,TRAMO DE VIA,1.000000,1900-01-01 01:00:00,MADRUGADA,6,1,2015
1,79,BARBOSA,07/01/2015,03:00:00,MIERCOLES,CHOQUE,HATILLO-BELLO KM 18+00,HERIDOS,PARQUE PRINCIPAL,SIN INFORMACIÓN,TRAMO DE VIA,3.000000,1900-01-01 03:00:00,MADRUGADA,3,1,2015
2,79,BARBOSA,04/01/2015,04:01:00,DOMINGO,CHOQUE,KM 16+395 METROS BARBOSA-HATILLO,HERIDOS,VIA EL HATILLO-CISNEROS,SIN INFORMACIÓN,TRAMO DE VIA,4.016667,1900-01-01 04:00:00,MADRUGADA,7,1,2015
3,79,BARBOSA,05/01/2015,12:00:00,LUNES,CHOQUE,BELLO- DON MATIAS KM 9+600,HERIDOS,EL HATILLO,SIN INFORMACIÓN,TRAMO DE VIA,12.000000,1900-01-01 12:00:00,TARDE,1,1,2015
4,79,BARBOSA,11/01/2015,01:30:00,DOMINGO,CHOQUE,LOS ABUELOS,DAÑOS,PARQUE PRINCIPAL,SIN INFORMACIÓN,TRAMO DE VIA,1.500000,1900-01-01 01:00:00,MADRUGADA,7,1,2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203430,631,SABANETA,30/08/2018,18:45:00,JUEVES,CHOQUE,CL 77 SUR CR 45,HERIDOS,SIN INFORMACIÓN,SIN INFORMACIÓN,SIN INFORMACIÓN,18.750000,1900-01-01 18:00:00,NOCHE,4,8,2018
203431,631,SABANETA,30/08/2018,18:45:00,JUEVES,CHOQUE,CL 77 SUR CR 45,HERIDOS,SIN INFORMACIÓN,SIN INFORMACIÓN,SIN INFORMACIÓN,18.750000,1900-01-01 18:00:00,NOCHE,4,8,2018
203432,631,SABANETA,31/08/2018,17:00:00,VIERNES,CAIDA OCUPANTE,CL 52 SUR CR 44,HERIDOS,SIN INFORMACIÓN,SIN INFORMACIÓN,SIN INFORMACIÓN,17.000000,1900-01-01 17:00:00,TARDE,5,8,2018
203433,631,SABANETA,31/08/2018,18:10:00,VIERNES,VOLCAMIENTO,CR 49 CL 51 SUR,HERIDOS,SIN INFORMACIÓN,SIN INFORMACIÓN,SIN INFORMACIÓN,18.166667,1900-01-01 18:00:00,NOCHE,5,8,2018


In [64]:
# CELDA 6: analisis_rapido (gráficas principales)
def analisis_rapido(df: pd.DataFrame):
    print("\n📄 ANÁLISIS EXPLORATORIO RÁPIDO")
    required = ["GRAVEDAD_ACCIDENTE", "JORNADA", "CLASE", "COMUNA"]
    missing = [c for c in required if c not in df.columns]
    if missing: raise KeyError(f"Faltan columnas: {missing}")

    format_torta(df["GRAVEDAD_ACCIDENTE"].value_counts(),
                 "Distribución por Gravedad de Accidentes",
                 OUT_DIR / "Accidentes_Gravedad_SVA.jpg")

    format_barra(df["JORNADA"].value_counts(),
                 "Cantidad de Accidentes por Jornada",
                 "Número de Accidentes", "Franja Horaria",
                 OUT_DIR / "Accidentes_Jornada_SVA.jpg")

    df_clase = df[df["CLASE"].str.upper() != "SIN INFORMACIÓN"]
    format_barra(df_clase["CLASE"].value_counts().head(10),
                 "Cantidad de Accidentes por Clase",
                 "Número de Accidentes", "Tipo de Accidente",
                 OUT_DIR / "Accidentes_Clase_SVA.jpg")

    df_comuna = df[df["COMUNA"].str.upper() != "SIN INFORMACIÓN"]
    format_barra(df_comuna["COMUNA"].value_counts().head(10),
                 "Top 10 - Accidentes por Comuna",
                 "Número de Accidentes", "Comuna",
                 OUT_DIR / "Accidentes_Comuna_SVA.jpg")

    df_jornada = df["JORNADA"].value_counts()
    format_barra(df_jornada["JORNADA"].value_counts().head(10),
                 "Jornada con más Accidentes",
                 "Número de Accidentes", "Jornada",
                 OUT_DIR / "Accidentes_JORNADA_SVA.jpg")

    df_año = df["AÑO"].value_counts()
    format_barra(df_año["AÑO"].value_counts().head(10),
                 "Años con más Accidentes",
                 "Número de Accidentes", "Años",
                 OUT_DIR / "Accidentes_AÑOS_SVA.jpg")

    print(f"✔️  Gráficas generadas en: {OUT_DIR}")
    print("="*60)


In [66]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
# CELDA 7: preparar_datos (preprocesamiento idéntico)
def preparar_datos(data: pd.DataFrame):
    print("📄 PREPARANDO DATOS PARA RANDOM FOREST")
    df_local = data.copy()
    df_local["HERIDOS_MUERTOS"] = df_local["GRAVEDAD_ACCIDENTE"].str.upper().isin(["HERIDOS", "MUERTOS"]).astype(np.uint8)
    df_local["FIN_DE_SEMANA"] = df_local["NUM_DIA_SEMANA"].isin([6,7]).astype(np.uint8)

    numeric_features = ['NUM_MES', 'NUM_DIA_SEMANA', 'NUM_HORA', 'FIN_DE_SEMANA']
    categorical_features = ['CLASE', 'MUNICIPIO', 'COMUNA', 'JORNADA']

    # Asegurar existencia de columnas categóricas si faltan, crear con 'SIN INFORMACIÓN'
    for c in categorical_features:
        if c not in df_local.columns:
            df_local[c] = 'SIN INFORMACIÓN'

    X = df_local[numeric_features + categorical_features]
    y = df_local['HERIDOS_MUERTOS']

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y )

    preprocessor = ColumnTransformer([
        ('num', Pipeline([
            ('imputer', SimpleImputer(strategy='median')),
            ('scaler', StandardScaler())
        ]), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=True), categorical_features)
    ])

    X_train_preprocessed = preprocessor.fit_transform(X_train)
    print(f"✔️  Forma después del preprocesamiento: {X_train_preprocessed.shape}")
    return X_train, X_test, y_train, y_test, preprocessor

In [ ]:
# CELDA 8: entrenamiento, evaluación y utilidades del modelo
def entrenar_random_forest(X_train, X_test, y_train, y_test, preprocessor):
    print("\n🤖 ENTRENAMIENTO RANDOM FOREST")
    model = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('classifier', RandomForestClassifier(
            n_estimators=5,      # Solo 5 árboles como solicitado
            random_state=42,
            max_depth=5          # Limitamos profundidad para evitar overfitting
        ))
    ])

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    y_proba = model.predict_proba(X_test)[:, 1]

    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred, zero_division=0)
    rec = recall_score(y_test, y_pred, zero_division=0)
    f1 = f1_score(y_test, y_pred, zero_division=0)
    roc_auc = roc_auc_score(y_test, y_proba)

    print(f"Exactitud: {acc:.4f} | Precisión: {prec:.4f} | Sensibilidad: {rec:.4f} | F1: {f1:.4f} | AUC-ROC: {roc_auc:.4f}")

    cm = confusion_matrix(y_test, y_pred)
    fig, ax = plt.subplots(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap=sns.blend_palette(PALETTE_GREEN[::-1], as_cmap=True),
                xticklabels=['Solo Daños', 'Con Heridos'], yticklabels=['Solo Daños', 'Con Heridos'], ax=ax)
    ax.set_title('Matriz de Confusión - Clasificación de Accidentes', fontsize=16)
    save_fig(fig, OUT_DIR / "Matriz_Confusion_SVA.jpg")

    fpr, tpr, _ = roc_curve(y_test, y_proba)
    fig, ax = plt.subplots(figsize=(10, 8))
    ax.plot(fpr, tpr, color=PALETTE_GREEN[1], lw=2, label=f'ROC (AUC={roc_auc:.2f})')
    ax.plot([0, 1], [0, 1], 'k--', label='Aleatorio')
    ax.set_xlim([-0.00, 1.01]); ax.set_ylim([-0.00, 1.01])
    ax.set_title('Curva ROC - Clasificación de Severidad de Accidentes')
    ax.set_xlabel('Tasa Falsos Positivos'); ax.set_ylabel('Tasa Verdaderos Positivos')
    ax.legend(loc="lower right")
    save_fig(fig, OUT_DIR / "Curva_ROC_SVA.jpg")

    print(classification_report(y_test, y_pred, target_names=['Solo Daños', 'Con Heridos']))
    return model, {'accuracy': acc, 'precision': prec, 'recall': rec, 'f1': f1, 'roc_auc': roc_auc}

def guardar_modelo(modelo):
    path = MODEL_DIR / "Modelo_RandomForest_SVA.joblib"
    joblib.dump(modelo, path)
    print(f"✔️ Modelo guardado en: {path}")

def hacer_predicciones(modelo):
    nuevos = pd.DataFrame({
        'NUM_MES': [1,7,12,3],
        'NUM_DIA_SEMANA': [6,2,4,1],
        'NUM_HORA': [20,14,8,18],
        'FIN_DE_SEMANA': [1,0,0,0],
        'CLASE': ['CHOQUE','ATROPELLO','CHOQUE','VOLCAMIENTO'],
        'MUNICIPIO': ['MEDELLÍN']*4,
        'COMUNA': ['LAURELES ESTADIO','LA CANDELARIA','CASTILLA','ROBLEDO'],
        'JORNADA': ['NOCHE','TARDE','MAÑANA','TARDE']
    })
    preds = modelo.predict(nuevos)
    proba = modelo.predict_proba(nuevos)[:, 1]
    resultados = nuevos.copy()
    resultados['PREDICCION'] = np.where(preds==1,'CON HERIDOS','SOLO DAÑOS')
    resultados['PROBABILIDAD_HERIDOS'] = [f"{p:.1%}" for p in proba]
    resultados['RIESGO'] = np.select([proba>0.7, proba>0.5], ['ALTO','MEDIO'], default='BAJO')
    save_path = MODEL_DIR / "Predicciones_Nuevos_Accidentes.csv"
    resultados.to_csv(save_path, index=False, encoding='utf-8-sig')
    print(f"✔️ Predicciones guardadas en {save_path}")
    return resultados

def analizar_importancia_variables(modelo: Pipeline, preprocessor: ColumnTransformer):
    rf = modelo.named_steps['classifier']
    num = ['NUM_MES','NUM_DIA_SEMANA','NUM_HORA','FIN_DE_SEMANA']
    cat = ['CLASE','MUNICIPIO','COMUNA','JORNADA']
    ohe = preprocessor.named_transformers_['cat']
    try:
        cat_names = ohe.get_feature_names_out(cat)
    except:
        # compatibilidad sklearn antigua
        cat_names = ohe.get_feature_names(cat)
    names = num + list(cat_names)
    imp = rf.feature_importances_
    imp_df = pd.DataFrame({'Variable': names, 'Importancia': imp}).sort_values('Importancia', ascending=False)
    fig, ax = plt.subplots(figsize=(12,8))
    sns.barplot(x='Importancia', y='Variable', data=imp_df.head(10), palette=paleta_antioquia(10), ax=ax)
    ax.set_title('Top 10 Variables Más Importantes - Random Forest', fontsize=16)
    save_fig(fig, MODEL_DIR / "Importancia_Variables_RF.jpg")
    imp_df.to_csv(MODEL_DIR / "Importancia_Variables_RF.csv", index=False, encoding='utf-8-sig')
    print("✔️ Importancia de variables guardada.")
    return imp_df

In [ ]:
# CELDA 9: resumen ejecutivo y ejecución final controlada
def generar_resumen_final(df: pd.DataFrame, resultados: dict):
    resumen_path = MODEL_DIR / "Resumen_Ejecutivo_Modelo.txt"
    with open(resumen_path, "w", encoding="utf-8") as f:
        f.write("📄 RESUMEN EJECUTIVO DEL PROYECTO\n")
        f.write("="*60 + "\n\n")

        total_accidentes = len(df)
        accidentes_con_heridos = len(df[df['GRAVEDAD_ACCIDENTE'].str.upper().isin(['HERIDOS', 'MUERTOS'])])
        tasa_heridos = accidentes_con_heridos / total_accidentes if total_accidentes else 0

        f.write("📈 ESTADÍSTICAS GENERALES:\n")
        f.write(f"   • Total de accidentes analizados: {total_accidentes:,}\n")
        f.write(f"   • Accidentes con heridos/muertos: {accidentes_con_heridos:,}\n")
        f.write(f"   • Tasa de accidentes con heridos: {tasa_heridos:.2%}\n\n")

        f.write("🤖 RESULTADOS DEL MODELO RANDOM FOREST:\n")
        f.write(f"   • Exactitud: {resultados['accuracy']:.2%}\n")
        f.write(f"   • Precisión: {resultados['precision']:.2%}\n")
        f.write(f"   • Sensibilidad: {resultados['recall']:.2%}\n")
        f.write(f"   • F1-Score: {resultados['f1']:.4f}\n")
        f.write(f"   • AUC-ROC: {resultados['roc_auc']:.4f}\n\n")

        try:
            franja_peligrosa = (
                df.groupby('JORNADA')['GRAVEDAD_ACCIDENTE']
                .apply(lambda x: x.str.upper().isin(['HERIDOS', 'MUERTOS']).mean())
                .idxmax() )
            tipo_peligroso = (
                df.groupby('CLASE')['GRAVEDAD_ACCIDENTE']
                .apply(lambda x: x.str.upper().isin(['HERIDOS', 'MUERTOS']).mean())
                .idxmax() )
            comuna_peligrosa = (
                df.groupby('COMUNA')['GRAVEDAD_ACCIDENTE']
                .apply(lambda x: x.str.upper().isin(['HERIDOS', 'MUERTOS']).mean())
                .idxmax() )

            f.write("🔍 HALLAZGOS PRINCIPALES:\n")
            f.write(f"   • Franja horaria más peligrosa: {franja_peligrosa}\n")
            f.write(f"   • Tipo de accidente más peligroso: {tipo_peligroso}\n")
            f.write(f"   • Comuna con mayor tasa de heridos: {comuna_peligrosa}\n\n")

            f.write("💡 RECOMENDACIONES:\n")
            f.write(f"   1. Reforzar vigilancia en: {franja_peligrosa}\n")
            f.write(f"   2. Implementar campañas preventivas para: {tipo_peligroso}\n")
            f.write(f"   3. Focalizar recursos de control en: {comuna_peligrosa}\n")
            f.write("   4. Utilizar el modelo predictivo para priorizar zonas de riesgo.\n")
        except Exception as e:
            f.write(f"❌ No se pudieron generar hallazgos detallados: {e}\n")

    print(f"✔️ Resumen ejecutivo guardado en: {resumen_path}")

# EJECUCIÓN CONTROLADA (main)
if __name__ == "__main__":
    try:
        df_db = load_table(DB_PATH, TABLE_NAME)
        analisis_rapido(df_db)
        X_train, X_test, y_train, y_test, preprocessor = preparar_datos(df_db)
        modelo_rf, resultados = entrenar_random_forest(X_train, X_test, y_train, y_test, preprocessor)
        preds_df = hacer_predicciones(modelo_rf)
        guardar_modelo(modelo_rf)
        imp_df = analizar_importancia_variables(modelo_rf, preprocessor)
        generar_resumen_final(df_db, resultados)
        print("\n✔️ Flujo completo: ejecutado correctamente.")
    except Exception as e:
        print(f"❌ Error durante la ejecución: {e}")

In [ ]:
# CELDA 10: listar archivos clave generados
import glob
print("\nArchivos generados (jpg, db, csv, joblib, txt):")
for file in sorted(glob.glob("/content/**/*", recursive=True)):
    if any(file.lower().endswith(ext) for ext in [".jpg", ".db", ".csv", ".joblib", ".txt"]):
        print(" ", file)

In [ ]:
# CELDA B: crear streamlit_app.py
code = r'''
import streamlit as st
import pandas as pd
import numpy as np
from pathlib import Path
import plotly.express as px
import plotly.graph_objects as go
from sqlalchemy import create_engine
import joblib

# Config
PALETTE_GREEN = ["#267344", "#37A85B", "#A9E4B4"]
DB_PATH = "/content/Proyecto_Accidentalidad_Vial_Antioquia.db"
MODEL_DIR = Path("/content/Modelo_Predict")
OUT_DIR = Path("/content/Graficas_Salida")

@st.cache_data
def load_db(table_name="Accidentalidad_Vial_Antioquia"):
    engine = create_engine(f"sqlite:///{DB_PATH}")
    df = pd.read_sql(f"SELECT * FROM {table_name}", engine)
    return df

@st.cache_data
def load_model_metrics():
    rpt = MODEL_DIR / "reporte_metricas.csv"
    if rpt.exists():
        try:
            return pd.read_csv(rpt, index_col=0)
        except:
            return None
    return None

@st.cache_data
def load_importance():
    p = MODEL_DIR / "Importancia_Variables_RF.csv"
    if p.exists():
        return pd.read_csv(p)
    return None

@st.cache_data
def load_model():
    p = MODEL_DIR / "Modelo_RandomForest_SVA.joblib"
    if p.exists():
        return joblib.load(p)
    return None

st.set_page_config(layout="wide", page_title="Accidentalidad Vial - Antioquia", initial_sidebar_state="expanded")

st.sidebar.title("Filtros")
df = load_db()

# Sidebar filters
min_mes = int(df["NUM_MES"].min()) if "NUM_MES" in df.columns else 1
max_mes = int(df["NUM_MES"].max()) if "NUM_MES" in df.columns else 12
mes_range = st.sidebar.slider("Mes (rango)", min_mes, max_mes, (min_mes, max_mes))

comunas = sorted(df["COMUNA"].dropna().unique()) if "COMUNA" in df.columns else []
comuna_sel = st.sidebar.multiselect("Comuna", options=comunas, default=comunas[:5])

clases = sorted(df["CLASE"].dropna().unique()) if "CLASE" in df.columns else []
clase_sel = st.sidebar.multiselect("Clase de accidente", options=clases, default=clases[:5])

jornadas = sorted(df["JORNADA"].dropna().unique()) if "JORNADA" in df.columns else []
jornada_sel = st.sidebar.multiselect("Jornada", options=jornadas, default=jornadas)

# Apply filters
df_f = df.copy()
if "NUM_MES" in df_f.columns:
    df_f = df_f[df_f["NUM_MES"].between(mes_range[0], mes_range[1])]
if comuna_sel:
    df_f = df_f[df_f["COMUNA"].isin(comuna_sel)]
if clase_sel:
    df_f = df_f[df_f["CLASE"].isin(clase_sel)]
if jornada_sel:
    df_f = df_f[df_f["JORNADA"].isin(jornada_sel)]

# Top KPIs
st.title("Accidentalidad Vial - Antioquia")
col1, col2, col3, col4 = st.columns(4)
total = len(df_f)
heridos = df_f["GRAVEDAD_ACCIDENTE"].str.upper().isin(["HERIDOS", "MUERTOS"]).sum() if "GRAVEDAD_ACCIDENTE" in df_f.columns else 0
tasa = (heridos/total) if total else 0

col1.metric("Total registros", f"{total:,}")
col2.metric("Accidentes con heridos/muertos", f"{heridos:,}")
col3.metric("Tasa heridos", f"{tasa:.2%}")

# A: Distribución de gravedad (pie + barras por mes)
st.header("Distribución por gravedad")
if "GRAVEDAD_ACCIDENTE" in df_f.columns:
    dist = df_f["GRAVEDAD_ACCIDENTE"].value_counts().reset_index()
    dist.columns = ["GRAVEDAD", "COUNT"]
    fig_pie = px.pie(dist, values="COUNT", names="GRAVEDAD", color="GRAVEDAD",
                     color_discrete_sequence=PALETTE_GREEN)
    st.plotly_chart(fig_pie, use_container_width=True)

    # barras por mes
    if "NUM_MES" in df_f.columns:
        by_month = df_f.groupby(["NUM_MES", "GRAVEDAD_ACCIDENTE"]).size().reset_index(name="COUNT")
        fig_bar = px.bar(by_month, x="NUM_MES", y="COUNT", color="GRAVEDAD_ACCIDENTE",
                         color_discrete_sequence=PALETTE_GREEN, barmode="stack",
                         labels={"NUM_MES": "Mes"})
        st.plotly_chart(fig_bar, use_container_width=True)

# B: Jornada (barra + heatmap horas vs día)
st.header("Accidentes por jornada y hora")
if "JORNADA" in df_f.columns:
    jcounts = df_f["JORNADA"].value_counts().reset_index()
    jcounts.columns = ["JORNADA", "COUNT"]
    fig_j = px.bar(jcounts, x="COUNT", y="JORNADA", orientation="h", color="JORNADA",
                   color_discrete_sequence=PALETTE_GREEN)
    st.plotly_chart(fig_j, use_container_width=True)

if "NUM_DIA_SEMANA" in df_f.columns and "NUM_HORA" in df_f.columns:
    pivot = df_f.dropna(subset=["NUM_DIA_SEMANA","NUM_HORA"]).groupby(["NUM_DIA_SEMANA","NUM_HORA"]).size().reset_index(name="COUNT")
    pivot = pivot.pivot(index="NUM_DIA_SEMANA", columns="NUM_HORA", values="COUNT").fillna(0)
    fig_heat = go.Figure(data=go.Heatmap(z=pivot.values, x=pivot.columns, y=pivot.index, colorscale="Greens"))
    fig_heat.update_layout(xaxis_title="Hora (decimal)", yaxis_title="Día de la semana")
    st.plotly_chart(fig_heat, use_container_width=True)

# C: Top clases y comunas
st.header("Top clases y comunas")
if "CLASE" in df_f.columns:
    top_clase = df_f["CLASE"].value_counts().head(10).reset_index()
    top_clase.columns = ["CLASE","COUNT"]
    fig_cl = px.bar(top_clase, x="COUNT", y="CLASE", orientation="h", color="CLASE", color_discrete_sequence=PALETTE_GREEN)
    st.plotly_chart(fig_cl, use_container_width=True)

if "COMUNA" in df_f.columns:
    top_com = df_f["COMUNA"].value_counts().head(10).reset_index()
    top_com.columns = ["COMUNA","COUNT"]
    fig_co = px.bar(top_com, x="COUNT", y="COMUNA", orientation="h", color="COMUNA", color_discrete_sequence=PALETTE_GREEN)
    st.plotly_chart(fig_co, use_container_width=True)

# Modelado: métricas, matriz, ROC, importancia
st.header("Evaluación del modelo")
model = load_model()
metrics = load_model_metrics()
if metrics is not None:
    st.subheader("Reporte de métricas (csv)")
    st.dataframe(metrics)

imp = load_importance()
if imp is not None:
    st.subheader("Importancia de variables (top 10)")
    st.dataframe(imp.head(10))
    fig_imp = px.bar(imp.sort_values("Importancia", ascending=False).head(10), x="Importancia", y="Variable", orientation="h", color="Importancia", color_continuous_scale="Greens")
    st.plotly_chart(fig_imp, use_container_width=True)

# Mostrar matriz y ROC desde imágenes si existen
if (OUT_DIR / "Matriz_Confusion_SVA.jpg").exists():
    st.image(str(OUT_DIR / "Matriz_Confusion_SVA.jpg"), caption="Matriz de Confusión", use_column_width=True)
if (OUT_DIR / "Curva_ROC_SVA.jpg").exists():
    st.image(str(OUT_DIR / "Curva_ROC_SVA.jpg"), caption="Curva ROC", use_column_width=True)

# Predicciones de ejemplo
st.header("Predicciones de ejemplo")
preds_path = MODEL_DIR / "Predicciones_Nuevos_Accidentes.csv"
if preds_path.exists():
    preds = pd.read_csv(preds_path)
    st.dataframe(preds)
    st.download_button("Descargar predicciones (CSV)", data=preds.to_csv(index=False).encode("utf-8-sig"), file_name="Predicciones_Nuevos_Accidentes.csv")

# Resumen ejecutivo
st.header("Resumen ejecutivo")
res_path = MODEL_DIR / "Resumen_Ejecutivo_Modelo.txt"
if res_path.exists():
    with open(res_path, "r", encoding="utf-8") as f:
        txt = f.read()
    st.text_area("Resumen ejecutivo", txt, height=300)
    st.download_button("Descargar resumen (TXT)", data=txt.encode("utf-8"), file_name="Resumen_Ejecutivo_Modelo.txt")

# Descargas: BD y ZIP resultados
st.sidebar.subheader("Exportar")
if st.sidebar.button("Descargar BD SQLite"):
    with open(DB_PATH, "rb") as f:
        st.sidebar.download_button("Descargar BD", data=f, file_name="Proyecto_Accidentalidad_Vial_Antioquia.db")
'''
Path("streamlit_app.py").write_text(code, encoding="utf-8")
print("Archivo streamlit_app.py creado.")


In [ ]:
# kill posible proceso previo
try:
    ngrok.kill()
except:
    pass

# ejecutar streamlit en background
port = 8501
cmd = f"streamlit run streamlit_app.py --server.port {port} --server.headless true"
proc = subprocess.Popen(cmd.split(), stdout=subprocess.PIPE, stderr=subprocess.PIPE)

# abrir tunel
public_url = ngrok.connect(port).public_url
print("Streamlit corriendo en:", public_url)
print("Detener la app con ngrok.kill() o interrumpiendo la celda.")
